In [ ]:
!nvidia-smi

Fri Jul 29 16:11:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets -q
!pip install transformers -q -U
!pip install wandb -q -U

     |████████████████████████████████| 365 kB 8.3 MB/s 
ERROR: Operation cancelled by user
     |████████████████████████████████| 4.7 MB 7.7 MB/s 
     |████████████████████████████████| 6.6 MB 76.9 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 


In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
DS_HOME = '/content/gdrive/Shareddrives/PROJECT_ROOT_DIR/injury-icd-dataset'
CODE_HOME = '/content/gdrive/Shareddrives/PROJECT_ROOT_DIR/code'
MODEL_HOME = '/content/gdrive/Shareddrives/PROJECT_ROOT_DIR/models'
TMP_DIR = '/content/tmp'

if not os.path.exists(TMP_DIR):
    os.mkdir(TMP_DIR)

os.chdir(CODE_HOME)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import datasets
import sys, importlib
import random
import shutil

In [ ]:
valid_labels = "4-char"                          #@param ["4-char", "4-char-top50", "4-char-top10", "5-char", "4-and-5-char"]
experiment_name = "non-sup-4-char-train-on-full" #@param {type:"string",  allow-input: true} ["non-sup", "non-sup-pretrain", "non-sup-tune-after-pretrain", "non-sup-4and5-char-train-on-full", "non-sup-train-on-full"]
model_name = "michiyasunaga/BioLinkBERT-base"         #@param ["michiyasunaga/BioLinkBERT-base", "michiyasunaga/BioLinkBERT-large", "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"]
metric_for_best_model = "eval_f1_score_weighted"          #@param ["eval_f1_score_macro", "eval_f1_score_micro", "eval_f1_score_weighted", "eval_auc_score_macro", "eval_auc_score_micro", "eval_auc_score_weighted"]
num_epochs = "20"                 #@param [6, 10, 20, 30]
# Whether to train on both train and val dataset, and do final eval on holdout test set
train_on_full = True            #@param {type:"boolean"} 
evaluate_only = True           #@param {type:"boolean"}
learning_rate = "0.00002"        #@param [2e-5, 1e-5, 7e-6, 2e-6]
warmup_steps =  5000            #@param [1000, 2000, 3000, 5000]
per_device_train_batch_size = "16" #@param [16, 8, 6, 4]
per_device_eval_batch_size = "32" #@param [32, 16, 12, 8]
learning_rate = float(learning_rate)
warmup_steps = int(warmup_steps)
per_device_train_batch_size = int(per_device_train_batch_size)
per_device_eval_batch_size = int(per_device_eval_batch_size)
if "base" in model_name:
  print("The suggested batch size for train and eval of base-sized model on P100 GPU is 16 and 32")
else:
  print("The suggested batch size for train and eval of large-sized model on P100 GPU is 6 and 12")

The suggested batch size for train and eval of large-sized model on P100 GPU is 6 and 12


In [ ]:
!python train.py --model_name=$model_name \
                 --data_dir=$DS_HOME \
                 --model_dir=$MODEL_HOME \
                 --experiment_name=$experiment_name \
                 --valid_labels=$valid_labels \
                 --is_evaluate=$evaluate_only \
                 --train_on_full=$train_on_full \
                 --num_train_epochs=$num_epochs \
                 --metric_for_best_model=$metric_for_best_model \
                 --learning_rate=$learning_rate \
                 --warmup_steps=$warmup_steps \
                 --per_device_train_batch_size=$per_device_train_batch_size \
                 --per_device_eval_batch_size=$per_device_eval_batch_size


In [ ]:
drive.flush_and_unmount()